<a href="https://colab.research.google.com/github/Rohith-Bollam/new-project/blob/main/Instagram_Post_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langgraph langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.7 MB/s eta 0:00:00


In [3]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [1]:
from google.colab import userdata
API_KEY=userdata.get('GOOGLE_API_KEY_1')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [4]:
from langgraph.graph import MessageGraph,END
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [5]:
writer_prompt = ChatPromptTemplate.from_template(
    "Write a Instagram post about this topic: {topic}"
)

writer_chain = writer_prompt | model

reviewer_prompt = ChatPromptTemplate.from_template(
    "Review the following Instagram post and suggest improvements: {post}"
)

reviewer_chain = reviewer_prompt | model

In [6]:
graph = MessageGraph()

def writer_node(state):
  topic = state[-1].content
  post = writer_chain.invoke({"topic": topic})
  print(type(post))
  return state + [AIMessage(content = post.content)]

def reviewer_node(state):
  post = state[-1].content
  review = reviewer_chain.invoke({"post": post})
  return state + [AIMessage(content = review.content)]

graph.add_node("Writer", writer_node)
graph.add_node("Reviewer", reviewer_node)
graph.set_entry_point("Writer")

def condition_check(state):
  print("From condition_check")
  if len(state)>=5:
    print("Printing from If")
    return END
  else:
    print("Printing from Else")
    return "Reviewer"

graph.add_conditional_edges("Writer", condition_check)
graph.add_edge("Reviewer", "Writer")

app = graph.compile()

In [7]:
response = app.invoke([HumanMessage(content = "Best side hustles for students")])
for message in response:
  print(message.content)

<class 'langchain_core.messages.ai.AIMessage'>
From condition_check
Printing from Else
<class 'langchain_core.messages.ai.AIMessage'>
From condition_check
Printing from Else
<class 'langchain_core.messages.ai.AIMessage'>
From condition_check
Printing from If
Best side hustles for students
Okay, here's an Instagram post draft about the best side hustles for students, designed to be engaging and informative:

**Option 1: (Focus on Variety & Quick Wins)**

```
✨ Student Hustle Alert! ✨

Need extra cash for textbooks, nights out, or that dream trip? ✈️ Ditch the ramen diet and explore these awesome side hustles that fit around your study schedule:

*   **Freelance Writing/Editing:** Sharpen those skills and get paid for your words! ✍️
*   **Online Tutoring:** Share your knowledge and help others succeed. 📚
*   **Social Media Management (for small businesses):** Are you a social media guru? Brands will pay for your skills! 📱
*   **Delivery Driver (Uber Eats, DoorDash):** Flexible hours and 